In [55]:
%pip install python-dotenv
import boto3
import sagemaker
import time
from dotenv import load_dotenv
import os

Note: you may need to restart the kernel to use updated packages.


In [59]:
load_dotenv()

!aws codebuild import-source-credentials --generate-cli-skeleton

{
    "username": "",
    "token": "",
    "serverType": "GITHUB",
    "authType": "OAUTH",
    "shouldOverwrite": true
}


In [62]:
codebuild_client = boto3.client("codebuild")

personal_access_token = os.getenv("PERSONAL_ACCESS_TOKEN")

environment_variables = [
    {
        "name": "MODEL_EXECUTION_ROLE_ARN",
        "value": sagemaker.get_execution_role(),
        "type": "PLAINTEXT",
    },
    {
        "name": "SOURCE_MODEL_PACKAGE_GROUP_NAME",
        "value": "index-predictor-model-group",
        "type": "PLAINTEXT",
    },
    {
        "name": "ARTIFACT_BUCKET",
        "value": "team1-index-predictor-bucket",
        "type": "PLAINTEXT",
    },
    {
        "name": "EXPORT_TEMPLATE_NAME",
        "value": "endpoint-config.json",
        "type": "PLAINTEXT",
    },
]

project_name = "index-predictor-codebuild"
source_location = "https://github.com/mlops-hs-team1/index-predictor"

response = codebuild_client.create_project(
    name=project_name,
    source={
        "type": "GITHUB",
        "location": source_location,
        "buildspec": "buildspec.yml",
    },
    artifacts={
        "type": "S3",
        "location": "team1-index-predictor-bucket",
        "name": "artifacts",
    },
    environment={
        "type": "LINUX_CONTAINER",
        "image": "aws/codebuild/standard:5.0",
        "computeType": "BUILD_GENERAL1_SMALL",
        "environmentVariables": environment_variables,
    },
    serviceRole=sagemaker.get_execution_role(),
    timeoutInMinutes=60,
)

print("CodeBuild project created successfully:", response)

InvalidInputException: An error occurred (InvalidInputException) when calling the CreateProject operation: No Access token found, please visit AWS CodeBuild console to connect to GitHub

In [63]:
project_name = "index-predictor-codebuild"
response = codebuild_client.start_build(
    projectName=project_name
)

print("Build started successfully:", response)

Build started successfully: {'build': {'id': 'index-predictor-codebuild:361ef609-dd3a-41a5-b6c3-eec26962b50b', 'arn': 'arn:aws:codebuild:eu-central-1:567821811420:build/index-predictor-codebuild:361ef609-dd3a-41a5-b6c3-eec26962b50b', 'buildNumber': 2, 'startTime': datetime.datetime(2024, 6, 25, 8, 22, 30, 63000, tzinfo=tzlocal()), 'currentPhase': 'QUEUED', 'buildStatus': 'IN_PROGRESS', 'projectName': 'index-predictor-codebuild', 'phases': [{'phaseType': 'SUBMITTED', 'phaseStatus': 'SUCCEEDED', 'startTime': datetime.datetime(2024, 6, 25, 8, 22, 30, 63000, tzinfo=tzlocal()), 'endTime': datetime.datetime(2024, 6, 25, 8, 22, 30, 114000, tzinfo=tzlocal()), 'durationInSeconds': 0}, {'phaseType': 'QUEUED', 'startTime': datetime.datetime(2024, 6, 25, 8, 22, 30, 114000, tzinfo=tzlocal())}], 'source': {'type': 'GITHUB', 'location': 'https://github.com/mlops-hs-team1/index-predictor', 'buildspec': 'buildspec.yml', 'reportBuildStatus': False, 'insecureSsl': False}, 'secondarySources': [], 'seconda

In [67]:
# Check build status
build_id = response['build']['id']
build_status = codebuild_client.batch_get_builds(ids=[build_id])
print("Build status:", build_status)

Build status: {'builds': [{'id': 'index-predictor-codebuild:361ef609-dd3a-41a5-b6c3-eec26962b50b', 'arn': 'arn:aws:codebuild:eu-central-1:567821811420:build/index-predictor-codebuild:361ef609-dd3a-41a5-b6c3-eec26962b50b', 'buildNumber': 2, 'startTime': datetime.datetime(2024, 6, 25, 8, 22, 30, 63000, tzinfo=tzlocal()), 'endTime': datetime.datetime(2024, 6, 25, 8, 22, 38, 848000, tzinfo=tzlocal()), 'currentPhase': 'COMPLETED', 'buildStatus': 'FAILED', 'projectName': 'index-predictor-codebuild', 'phases': [{'phaseType': 'SUBMITTED', 'phaseStatus': 'SUCCEEDED', 'startTime': datetime.datetime(2024, 6, 25, 8, 22, 30, 63000, tzinfo=tzlocal()), 'endTime': datetime.datetime(2024, 6, 25, 8, 22, 30, 114000, tzinfo=tzlocal()), 'durationInSeconds': 0}, {'phaseType': 'QUEUED', 'phaseStatus': 'SUCCEEDED', 'startTime': datetime.datetime(2024, 6, 25, 8, 22, 30, 114000, tzinfo=tzlocal()), 'endTime': datetime.datetime(2024, 6, 25, 8, 22, 30, 745000, tzinfo=tzlocal()), 'durationInSeconds': 0}, {'phaseTyp

In [6]:
sagemaker.get_execution_role()

'arn:aws:iam::567821811420:role/service-role/AmazonSageMaker-ExecutionRole-20230619T084765'

In [17]:
!aws cloudformation deploy --template-file /home/sagemaker-user/src/deployment/endpoint-config.yml --stack-name sagemaker-v3--parameter-overrides ModelExecutionRoleArn=arn:aws:iam::567821811420:role/service-role/AmazonSageMaker-ExecutionRole-20230619T084765


Unknown options: ModelExecutionRoleArn=arn:aws:iam::567821811420:role/service-role/AmazonSageMaker-ExecutionRole-20230619T084765


In [18]:
!aws cloudformation describe-stack-events --stack-name sagemaker-v2

{
    "StackEvents": [
        {
            "StackId": "arn:aws:cloudformation:eu-central-1:567821811420:stack/sagemaker-v2/747b8130-32c2-11ef-bc85-029d88de1407",
            "EventId": "7b5cac90-32c2-11ef-95bf-06840981f8af",
            "StackName": "sagemaker-v2",
            "LogicalResourceId": "sagemaker-v2",
            "PhysicalResourceId": "arn:aws:cloudformation:eu-central-1:567821811420:stack/sagemaker-v2/747b8130-32c2-11ef-bc85-029d88de1407",
            "ResourceType": "AWS::CloudFormation::Stack",
            "Timestamp": "2024-06-25T07:13:54.636000+00:00",
            "ResourceStatus": "ROLLBACK_COMPLETE"
        },
        {
            "StackId": "arn:aws:cloudformation:eu-central-1:567821811420:stack/sagemaker-v2/747b8130-32c2-11ef-bc85-029d88de1407",
            "EventId": "Model-DELETE_COMPLETE-2024-06-25T07:13:54.235Z",
            "StackName": "sagemaker-v2",
            "LogicalResourceId": "Model",
            "PhysicalResourceId": "",
            "ResourceType"